In [1]:
import joblib
import pickle
import pandas as pd
import numpy as np

In [2]:
import sklearn
print(sklearn.__version__)

1.2.2


In [3]:
with open("random_forest.joblib", "rb") as fr:
    model = joblib.load(fr);
with open("le_gu.pkl", "rb") as fr:
    le_gu = pickle.load(fr);
with open("le_dong.pkl", "rb") as fr:
    le_dong = pickle.load(fr);

In [18]:
file_path = 'apartment_transaction.txt'
df = pd.read_csv(file_path, encoding='utf-8', sep='|')
df

,dealDate,interestRate,gu,dong,exclusiveArea,floor,buildYear,dealAmount,apartmentName,jibunAddress
0,2024-10-19,3.25,강남구,대치동,84.580,1,2000,250000,대치삼성,강남구 대치동 1014-3
1,2024-10-29,3.25,강남구,청담동,90.197,9,2011,290000,청담자이,강남구 청담동 134-38
2,2024-10-19,3.25,강남구,개포동,59.888,2,2019,240000,래미안블레스티지,강남구 개포동 1280
3,2024-10-26,3.25,강남구,대치동,84.950,20,2008,320000,대치아이파크,강남구 대치동 888
4,2024-10-23,3.25,강남구,대치동,85.000,8,1999,254500,대치현대,강남구 대치동 974
...,...,...,...,...,...,...,...,...,...,...
1288183,2006-01-21,3.75,중랑구,신내동,39.840,4,1996,9700,신내6대주,중랑구 신내동 650
1288184,2006-01-18,3.75,중랑구,신내동,39.840,6,1996,9560,신내6대주,중랑구 신내동 650
1288185,2006-01-03,3.75,중랑구,망우동,51.690,5,2002,9000,현대,중랑구 망우동 508-17
1288186,2006-01-04,3.75,중랑구,신내동,59.400,7,1993,12750,동성1,중랑구 신내동 397


In [19]:
X = df.drop(['dealAmount', 'apartmentName', 'jibunAddress'], axis=1)

X['dealDate'] = pd.to_datetime(X['dealDate'])
X['year'] = X['dealDate'].dt.year
X['month'] = X['dealDate'].dt.month
X['day'] = X['dealDate'].dt.day

X['month_sin'] = np.sin(2*np.pi*X['month'] / 12)
X['month_cos'] = np.cos(2*np.pi*X['month'] / 12)
X['day_sin'] = np.sin(2*np.pi*X['day'] / 12)
X['day_cos'] = np.cos(2*np.pi*X['day'] / 12)

X = X.drop(['dealDate', 'month', 'day'], axis=1)

X['gu'] = le_gu.transform(X['gu'])
X['dong'] = le_dong.transform(X['dong'])

X

,interestRate,gu,dong,exclusiveArea,floor,buildYear,year,month_sin,month_cos,day_sin,day_cos
0,3.25,0,52,84.580,1,2000,2024,-0.866025,0.500000,-5.000000e-01,-8.660254e-01
1,3.25,0,259,90.197,9,2011,2024,-0.866025,0.500000,5.000000e-01,-8.660254e-01
2,3.25,0,8,59.888,2,2019,2024,-0.866025,0.500000,-5.000000e-01,-8.660254e-01
3,3.25,0,52,84.950,20,2008,2024,-0.866025,0.500000,8.660254e-01,5.000000e-01
4,3.25,0,52,85.000,8,1999,2024,-0.866025,0.500000,-5.000000e-01,8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...
1288183,3.75,23,158,39.840,4,1996,2006,0.500000,0.866025,-1.000000e+00,-4.286264e-16
1288184,3.75,23,158,39.840,6,1996,2006,0.500000,0.866025,3.673940e-16,-1.000000e+00
1288185,3.75,23,83,51.690,5,2002,2006,0.500000,0.866025,1.000000e+00,6.123234e-17
1288186,3.75,23,158,59.400,7,1993,2006,0.500000,0.866025,8.660254e-01,-5.000000e-01


In [20]:
y_predict = model.predict(X)
df['prediction'] = y_predict
df['difference'] = (df['prediction'] - df['dealAmount']) / df['dealAmount']

mean_diff = df['difference'].mean()
std_diff = df['difference'].std()

lower_bound = mean_diff - 1.96*std_diff
upper_bound = mean_diff + 1.96*std_diff

df['reliable'] = df['difference'].between(lower_bound, upper_bound)
df

,dealDate,interestRate,gu,dong,exclusiveArea,floor,buildYear,dealAmount,apartmentName,jibunAddress,prediction,difference,reliable
0,2024-10-19,3.25,강남구,대치동,84.580,1,2000,250000,대치삼성,강남구 대치동 1014-3,249071.134921,-0.003715,True
1,2024-10-29,3.25,강남구,청담동,90.197,9,2011,290000,청담자이,강남구 청담동 134-38,301769.871795,0.040586,True
2,2024-10-19,3.25,강남구,개포동,59.888,2,2019,240000,래미안블레스티지,강남구 개포동 1280,237782.575758,-0.009239,True
3,2024-10-26,3.25,강남구,대치동,84.950,20,2008,320000,대치아이파크,강남구 대치동 888,305501.333333,-0.045308,True
4,2024-10-23,3.25,강남구,대치동,85.000,8,1999,254500,대치현대,강남구 대치동 974,236889.206317,-0.069198,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288183,2006-01-21,3.75,중랑구,신내동,39.840,4,1996,9700,신내6대주,중랑구 신내동 650,9616.922627,-0.008565,True
1288184,2006-01-18,3.75,중랑구,신내동,39.840,6,1996,9560,신내6대주,중랑구 신내동 650,9585.363680,0.002653,True
1288185,2006-01-03,3.75,중랑구,망우동,51.690,5,2002,9000,현대,중랑구 망우동 508-17,10014.583333,0.112731,True
1288186,2006-01-04,3.75,중랑구,신내동,59.400,7,1993,12750,동성1,중랑구 신내동 397,13666.052335,0.071847,True


In [21]:
df['dealDate'] = pd.to_datetime(df['dealDate'])
df.sort_values(by='dealDate', ascending=False, inplace=True)

In [22]:
std_diff

0.2008552902593486

In [ ]:
df.to_csv('results.csv', index=False, sep='|', encoding='utf-8')